## 这是用来实现对npy保存的深度图像的检查,以及检查对齐效果的工具

In [33]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [34]:

from PIL import Image, ImageDraw, ImageFont

def put_chinese_text(img, text, position, font_size=30, color=(255, 255, 255)):
    """
    在图像上绘制中文文本
    
    参数:
        img: OpenCV图像 (numpy数组)
        text: 要绘制的中文文本
        position: 文本位置 (x, y)
        font_size: 字体大小
        color: 文本颜色 (B, G, R)
    
    返回:
        添加了中文文本的图像
    """
    # 将OpenCV图像转换为PIL格式 (BGR转RGB)
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    
    # 尝试加载不同的中文字体
    font_paths = [
        "/usr/share/fonts/truetype/wqy/wqy-microhei.ttc",  # Linux: 文泉驿微米黑
        "C:/Windows/Fonts/simhei.ttf",                     # Windows: 黑体
        "C:/Windows/Fonts/msyh.ttc",                      # Windows: 微软雅黑
        "C:/Windows/Fonts/simsun.ttc",                     # Windows: 宋体
        "/System/Library/Fonts/PingFang.ttc"               # macOS: 苹方
    ]
    
    font = None
    for font_path in font_paths:
        try:
            font = ImageFont.truetype(font_path, font_size)
            break
        except:
            continue
    
    # 如果系统字体都找不到，使用默认字体（可能不支持中文）
    if font is None:
        font = ImageFont.load_default()
        print(" 未找到中文字体，使用默认字体（可能不支持中文）")
    
    # 绘制中文文本
    draw.text(position, text, font=font, fill=color)
    
    # 转换回OpenCV格式 (RGB转BGR)
    img_with_text = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img_with_text

In [35]:
def check_alignment_quality(depth_data, color_image, alignment_threshold=0.6):
    """
    检查深度图与彩色图的对齐质量
    
    参数:
        depth_data (numpy.ndarray): 深度图数据
        color_image (numpy.ndarray): 彩色图像
        alignment_threshold (float): 对齐质量阈值，默认0.6（0-1之间）
        
    返回:
        tuple: (对齐得分, 对齐状态, 可视化图像)
    """
    # 检查图像尺寸是否匹配
    if depth_data.shape[:2] != color_image.shape[:2]:
        print(f"⚠️ 尺寸不匹配: 深度图{depth_data.shape} vs 彩色图{color_image.shape}")
        return 0.0, "尺寸不匹配", None
    
    # 将彩色图转为灰度图用于边缘检测
    if len(color_image.shape) == 3:
        gray_color = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    else:
        gray_color = color_image
    
    # 处理深度图：转换为8位并增强对比度
    depth_vis = depth_data.copy()
    depth_vis[np.isnan(depth_vis)] = 0
    depth_vis[depth_vis == 0] = np.nanmax(depth_vis)
    depth_8bit = cv2.convertScaleAbs(depth_vis, alpha=0.03)
    depth_8bit = cv2.equalizeHist(depth_8bit)  # 直方图均衡化增强细节
    
    # 使用Canny边缘检测
    color_edges = cv2.Canny(gray_color, 50, 150)
    depth_edges = cv2.Canny(depth_8bit, 50, 150)
    
    # 计算边缘重合度
    overlapping_edges = cv2.bitwise_and(color_edges, depth_edges)
    
    total_color_edges = np.sum(color_edges > 0)
    total_overlap = np.sum(overlapping_edges > 0)
    
    if total_color_edges == 0:
        alignment_score = 0.0
    else:
        alignment_score = total_overlap / total_color_edges
    
    # 判断对齐状态
    if alignment_score >= alignment_threshold:
        alignment_status = "良好对齐"
    elif alignment_score >= 0.3:
        alignment_status = "部分对齐"
    else:
        alignment_status = "未对齐"
    
    # 创建可视化图像
    color_edges_rgb = cv2.cvtColor(color_edges, cv2.COLOR_GRAY2BGR)
    depth_edges_rgb = cv2.cvtColor(depth_edges, cv2.COLOR_GRAY2BGR)
    overlap_rgb = cv2.cvtColor(overlapping_edges, cv2.COLOR_GRAY2BGR)
    
    # 用颜色标记不同类型的边缘
    color_edges_rgb[color_edges > 0] = [0, 255, 0]  # 绿色：彩色图边缘
    depth_edges_rgb[depth_edges > 0] = [255, 0, 0]   # 蓝色：深度图边缘
    overlap_rgb[overlapping_edges > 0] = [0, 0, 255] # 红色：重合边缘
    
    # 叠加显示
    overlay_image = cv2.addWeighted(color_edges_rgb, 0.5, depth_edges_rgb, 0.5, 0)
    overlay_image = cv2.addWeighted(overlay_image, 0.7, overlap_rgb, 0.3, 0)
    
    # 添加状态文本
    status_text = f"对齐状态: {alignment_status} (得分: {alignment_score:.3f})"
    color_map = (0, 255, 0) if alignment_status == "良好对齐" else (0, 255, 255) if alignment_status == "部分对齐" else (0, 0, 255)
    
    cv2.putText(overlay_image, status_text, (10, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_map, 2)
    cv2.putText(overlay_image, "绿色:彩色边缘 蓝色:深度边缘 红色:重合边缘", (10, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    return alignment_score, alignment_status, overlay_image

def load_and_visualize_depth_npy(depth_dir):
    """
    批量加载和可视化深度图(.npy文件)，增加对齐检查功能
    
    参数:
        depth_dir (str): 深度图文件目录路径
        
    返回:
        tuple: (深度图数据列表, 对齐状态统计)
    """
    depth_files = [f for f in os.listdir(depth_dir) if f.endswith('.npy')]
    depth_files.sort()
    
    depth_data_list = []
    alignment_stats = []  # 存储每张图的对齐状态
    
    print(f"找到 {len(depth_files)} 个深度图文件")
    
    for i, filename in enumerate(depth_files):
        file_path = os.path.join(depth_dir, filename)
        
        try:
            depth_data = np.load(file_path)
            depth_data_list.append(depth_data)
            
            # 打印基本信息
            valid_depth = depth_data[~np.isnan(depth_data) & (depth_data != 0)]
            if len(valid_depth) > 0:
                depth_range = f"[{np.min(valid_depth):.3f}, {np.max(valid_depth):.3f}]米"
            else:
                depth_range = "无有效深度值"
            
            print(f"深度图 {filename}: 形状={depth_data.shape}, 数据类型={depth_data.dtype}, 值范围={depth_range}")
            
            # 深度图可视化
            depth_vis = depth_data.copy()
            depth_vis[np.isnan(depth_vis)] = 0
            depth_vis[depth_vis == 0] = np.nanmax(depth_vis) if np.any(~np.isnan(depth_data)) else 1
            
            depth_normalized = cv2.normalize(depth_vis, None, 0, 255, cv2.NORM_MINMAX)
            depth_8bit = np.uint8(depth_normalized)
            depth_colormap = cv2.applyColorMap(depth_8bit, cv2.COLORMAP_JET)
            
            # 标记无效区域
            invalid_mask = np.isnan(depth_data) | (depth_data == 0)
            depth_colormap[invalid_mask] = [0, 0, 0]
            
            # 显示深度图
            cv2.imshow(f'Depth Map: {filename}', depth_colormap)
            cv2.waitKey(50)
            
            # 保存可视化结果
            output_path = os.path.join(depth_dir, f"vis_{filename.replace('.npy', '.png')}")
            cv2.imwrite(output_path, depth_colormap)
            
        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")
            alignment_stats.append(("错误", 0.0))
    
    return depth_data_list, alignment_stats

def load_and_visualize_color_png(color_dir):
    """
    批量加载和可视化彩色图像(.png文件)
    
    参数:
        color_dir (str): 彩色图像文件目录路径
        
    返回:
        list: 包含所有彩色图像的列表
    """
    color_files = [f for f in os.listdir(color_dir) if f.endswith('.png')]
    color_files.sort()
    
    color_images_list = []
    
    print(f"找到 {len(color_files)} 个彩色图像文件")
    
    for i, filename in enumerate(color_files):
        file_path = os.path.join(color_dir, filename)
        
        try:
            color_image = cv2.imread(file_path)
            
            if color_image is None:
                print(f"无法读取文件: {filename}")
                continue
                
            color_images_list.append(color_image)
            print(f"彩色图 {filename}: 形状={color_image.shape}, 数据类型={color_image.dtype}")
            
            # 显示彩色图像
            cv2.imshow(f'Color Image: {filename}', color_image)
            cv2.waitKey(50)
            
        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")
    
    return color_images_list

def visualize_alignment_comparison(depth_data_list, color_images_list, depth_dir, color_dir):
    """
    可视化深度图与彩色图的对齐效果对比，并判断对齐状态
    
    参数:
        depth_data_list (list): 深度图数据列表
        color_images_list (list): 彩色图像列表
        depth_dir (str): 深度图目录
        color_dir (str): 彩色图目录
    """
    print("\n开始对齐效果对比和判断...")
    
    min_length = min(len(depth_data_list), len(color_images_list))
    alignment_results = []
    
    # 检查文件命名是否暗示了对齐状态（常见对齐命名模式）
    depth_files = [f for f in os.listdir(depth_dir) if f.endswith('.npy')]
    depth_files.sort()
    
    # 检测可能的对齐命名模式
    aligned_naming = any('aligned' in f.lower() or 'registered' in f.lower() for f in depth_files)
    if aligned_naming:
        print("📝 检测到文件名中包含'aligned'或'registered'，可能已进行对齐处理")
    
    for i in range(min_length):
        depth_data = depth_data_list[i]
        color_image = color_images_list[i]
        
        print(f"\n检查图像对 #{i}:")
        
        # 检查对齐质量
        alignment_score, alignment_status, alignment_viz = check_alignment_quality(depth_data, color_image)
        alignment_results.append((i, alignment_score, alignment_status))
        
        print(f"对齐状态: {alignment_status} (得分: {alignment_score:.3f})")
        
        # 根据对齐状态给出建议
        if alignment_status == "未对齐":
            print("❌ 建议: 深度图与彩色图未对齐，建议使用RealSense的align功能重新采集或处理")
        elif alignment_status == "部分对齐":
            print("⚠️ 建议: 对齐效果一般，可能影响后续应用精度")
        else:
            print("✅ 对齐效果良好")
        
        # 调整深度图尺寸以匹配彩色图（如果需要）
        if depth_data.shape[:2] != color_image.shape[:2]:
            print("🔄 调整深度图尺寸以匹配彩色图...")
            depth_resized = cv2.resize(depth_data, 
                                     (color_image.shape[1], color_image.shape[0]))
        else:
            depth_resized = depth_data
        
        # 创建深度图可视化
        depth_vis = depth_resized.copy()
        depth_vis[np.isnan(depth_vis)] = 0
        depth_vis[depth_vis == 0] = np.nanmax(depth_vis) if np.any(~np.isnan(depth_resized)) else 1
        
        depth_normalized = cv2.normalize(depth_vis, None, 0, 255, cv2.NORM_MINMAX)
        depth_8bit = np.uint8(depth_normalized)
        depth_colormap = cv2.applyColorMap(depth_8bit, cv2.COLORMAP_JET)
        
        # 创建对比图像（左右并排）
        comparison = np.hstack((color_image, depth_colormap))

        # 添加对齐状态信息
        status_color = (0, 255, 0) if alignment_status == "良好对齐" else (0, 255, 255) if alignment_status == "部分对齐" else (0, 0, 255)

        
        # 显示对比结果和对齐分析
        if alignment_viz is not None:
            cv2.imshow('Alignment Analysis', alignment_viz)
        
        cv2.imshow('Color vs Depth Alignment', comparison)
        
        # 保存对比结果
        comparison_path = os.path.join(depth_dir, f"comparison_{i:04d}.png")
        cv2.imwrite(comparison_path, comparison)
        
        if alignment_viz is not None:
            alignment_viz_path = os.path.join(depth_dir, f"alignment_analysis_{i:04d}.png")
            cv2.imwrite(alignment_viz_path, alignment_viz)
        
        print(f"已保存对比图: {comparison_path}")
        
        key = cv2.waitKey(2000)  # 显示2秒
        if key == ord('q'):  # 按q键退出
            break
    
    # 生成对齐统计报告
    generate_alignment_report(alignment_results, depth_dir)

def generate_alignment_report(alignment_results, output_dir):
    """
    生成对齐质量统计报告
    
    参数:
        alignment_results (list): 对齐结果列表
        output_dir (str): 输出目录
    """
    if not alignment_results:
        return
    
    print("\n" + "="*60)
    print("对齐质量统计报告")
    print("="*60)
    
    scores = [result[1] for result in alignment_results]
    statuses = [result[2] for result in alignment_results]
    
    avg_score = np.mean(scores)
    good_count = statuses.count("良好对齐")
    partial_count = statuses.count("部分对齐")
    poor_count = statuses.count("未对齐")
    
    print(f"总图像对数量: {len(alignment_results)}")
    print(f"平均对齐得分: {avg_score:.3f}")
    print(f"良好对齐: {good_count} 对 ({good_count/len(alignment_results)*100:.1f}%)")
    print(f"部分对齐: {partial_count} 对 ({partial_count/len(alignment_results)*100:.1f}%)")
    print(f"未对齐: {poor_count} 对 ({poor_count/len(alignment_results)*100:.1f}%)")
    
    # 总体建议
    if avg_score >= 0.6:
        print("✅ 总体对齐质量: 良好")
        print("💡 建议: 数据质量较好，可直接用于大多数RGB-D应用")
    elif avg_score >= 0.3:
        print("⚠️ 总体对齐质量: 一般")
        print("💡 建议: 考虑重新采集或使用对齐算法进行后处理")
    else:
        print("❌ 总体对齐质量: 较差")
        print("💡 建议: 强烈建议重新采集数据，确保启用RealSense的align功能")
    
    # 保存统计报告
    report_path = os.path.join(output_dir, "alignment_report.txt")
    with open(report_path, 'w') as f:
        f.write("深度图与彩色图对齐质量报告\n")
        f.write("="*40 + "\n")
        f.write(f"总图像对数量: {len(alignment_results)}\n")
        f.write(f"平均对齐得分: {avg_score:.3f}\n")
        f.write(f"良好对齐: {good_count} 对\n")
        f.write(f"部分对齐: {partial_count} 对\n")
        f.write(f"未对齐: {poor_count} 对\n")
    
    print(f"详细报告已保存至: {report_path}")


In [36]:

def main():
    """主函数"""
    base_path = "/home/qyw23ai/yolo-realsense_ws/2025_10_03_19_00_02"
    depth_dir = os.path.join(base_path, "depth")
    color_dir = os.path.join(base_path, "rgb")
    
    if not os.path.exists(depth_dir):
        print(f"错误: 深度图目录不存在 {depth_dir}")
        return
    
    if not os.path.exists(color_dir):
        print(f"错误: 彩色图目录不存在 {color_dir}")
        return
    
    print("开始批量处理深度图和彩色图...")
    
    # 批量处理深度图
    depth_data_list, _ = load_and_visualize_depth_npy(depth_dir)
    
    # 批量处理彩色图
    color_images_list = load_and_visualize_color_png(color_dir)
    
    # 对齐效果对比和判断
    if depth_data_list and color_images_list:
        visualize_alignment_comparison(depth_data_list, color_images_list, depth_dir, color_dir)
    else:
        print("错误: 无法加载深度图或彩色图数据")
    
    cv2.destroyAllWindows()
    print("\n处理完成！")

if __name__ == "__main__":
    main()

开始批量处理深度图和彩色图...
找到 1 个深度图文件
深度图 1.npy: 形状=(480, 640), 数据类型=float64, 值范围=[0.314, 12.789]米
找到 1 个彩色图像文件
彩色图 1.png: 形状=(480, 640, 3), 数据类型=uint8

开始对齐效果对比和判断...

检查图像对 #0:
对齐状态: 未对齐 (得分: 0.000)
❌ 建议: 深度图与彩色图未对齐，建议使用RealSense的align功能重新采集或处理
已保存对比图: /home/qyw23ai/yolo-realsense_ws/2025_10_03_19_00_02/depth/comparison_0000.png

对齐质量统计报告
总图像对数量: 1
平均对齐得分: 0.000
良好对齐: 0 对 (0.0%)
部分对齐: 0 对 (0.0%)
未对齐: 1 对 (100.0%)
❌ 总体对齐质量: 较差
💡 建议: 强烈建议重新采集数据，确保启用RealSense的align功能
详细报告已保存至: /home/qyw23ai/yolo-realsense_ws/2025_10_03_19_00_02/depth/alignment_report.txt

处理完成！
